## 1. Setup and Imports

In [ ]:
import sys
import os
sys.path.append('../structured_products')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from structured_products import Autocallable, ReverseConvertible, BarrierOption, RangeAccrual
from structured_products.pricing_engine import PricingEngine

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Setup complete!")

## 2. Payoff Diagrams

Visualize payoff profiles at maturity for different products.

In [ ]:
def plot_payoff_comparison(products, names, spot_range=None):
    """
    Plot payoff diagrams for multiple products.
    """
    if spot_range is None:
        spot_range = (60, 140)
    
    spots = np.linspace(spot_range[0], spot_range[1], 200)
    
    plt.figure(figsize=(14, 8))
    
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    
    for i, (product, name) in enumerate(zip(products, names)):
        payoffs = []
        for s in spots:
            # Create a simple path ending at s
            path = np.array([product.spot, s])
            payoff = product.payoff(path)
            payoffs.append(payoff)
        
        plt.plot(spots, payoffs, color=colors[i % len(colors)], 
                linewidth=2, label=name, alpha=0.8)
    
    # Add reference lines
    plt.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='Capital Protection')
    plt.axvline(x=100, color='gray', linestyle='--', alpha=0.5, label='Spot Price')
    
    plt.xlabel('Underlying Price at Maturity')
    plt.ylabel('Payoff')
    plt.title('Payoff Diagrams Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Create sample products
spot = 100
products = [
    Autocallable(spot=spot, strike=spot, barrier=85, coupon=0.05, maturity=3, 
                autocall_levels=[105, 110, 115]),
    ReverseConvertible(spot=spot, strike=spot, coupon=0.07, maturity=3, 
                      capital_protection=0.9),
    BarrierOption(spot=spot, strike=spot, barrier=90, maturity=3, 
                 option_type='call', barrier_type='up_and_in'),
    RangeAccrual(spot=spot, strike=spot, 
                ranges=[(90, 110), (85, 115), (80, 120)], 
                coupon_rates=[0.06, 0.06, 0.06], maturity=3)
]

names = ['Autocallable', 'Reverse Convertible', 'Barrier Option', 'Range Accrual']

plot_payoff_comparison(products, names)

## 3. Greeks Sensitivity Analysis

Visualize how Greeks change with underlying price and volatility.

In [ ]:
def plot_greeks_surface(product, param_range, param_name='spot', n_points=20):
    """
    Plot Greeks sensitivity surface.
    """
    param_values = np.linspace(param_range[0], param_range[1], n_points)
    greeks_data = {'delta': [], 'gamma': [], 'vega': [], 'theta': []}
    
    base_spot = product.spot
    base_vol = product.vol
    
    for param_val in param_values:
        if param_name == 'spot':
            product.spot = param_val
        elif param_name == 'vol':
            product.vol = param_val
        
        greeks = product.calculate_greeks(n_sim=1000)
        
        for greek in greeks_data.keys():
            greeks_data[greek].append(greeks[greek])
    
    # Reset parameters
    product.spot = base_spot
    product.vol = base_vol
    
    # Plot
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.ravel()
    
    greek_names = ['Delta', 'Gamma', 'Vega', 'Theta']
    
    for i, (greek, name) in enumerate(zip(greeks_data.keys(), greek_names)):
        axes[i].plot(param_values, greeks_data[greek], 'b-', linewidth=2)
        axes[i].set_xlabel(f'{param_name.title()}')
        axes[i].set_ylabel(name)
        axes[i].set_title(f'{name} vs {param_name.title()}')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Analyze Greeks for autocallable
autocall = products[0]
print("Greeks sensitivity for Autocallable:")
plot_greeks_surface(autocall, (80, 120), 'spot')
plot_greeks_surface(autocall, (0.1, 0.4), 'vol')

## 4. Scenario Analysis Heatmap

Show how product prices change under different market scenarios.

In [ ]:
def plot_scenario_heatmap(product, spot_shocks, vol_shocks):
    """
    Plot price sensitivity heatmap for different scenarios.
    """
    prices = np.zeros((len(vol_shocks), len(spot_shocks)))
    
    base_spot = product.spot
    base_vol = product.vol
    
    for i, vol_shock in enumerate(vol_shocks):
        for j, spot_shock in enumerate(spot_shocks):
            product.spot = base_spot * (1 + spot_shock)
            product.vol = base_vol * (1 + vol_shock)
            
            price = product.price_mc(n_sim=2000)
            prices[i, j] = price
    
    # Reset
    product.spot = base_spot
    product.vol = base_vol
    
    # Plot heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(prices, 
                xticklabels=[f'{s:.0%}' for s in spot_shocks],
                yticklabels=[f'{v:.0%}' for v in vol_shocks],
                annot=True, fmt='.3f', cmap='RdYlGn_r',
                cbar_kws={'label': 'Product Price'})
    
    plt.xlabel('Spot Price Shock')
    plt.ylabel('Volatility Shock')
    plt.title(f'Price Sensitivity Heatmap - {product.__class__.__name__}')
    plt.show()

# Scenario analysis for reverse convertible
rev_conv = products[1]
spot_shocks = np.linspace(-0.2, 0.2, 9)  # -20% to +20%
vol_shocks = np.linspace(-0.3, 0.3, 7)   # -30% to +30%

plot_scenario_heatmap(rev_conv, spot_shocks, vol_shocks)

## 5. Probability Distributions

Show the distribution of payoffs and key risk metrics.

In [ ]:
def plot_payoff_distribution(product, n_sim=5000):
    """
    Plot payoff distribution with risk metrics.
    """
    payoffs = []
    
    for _ in range(n_sim):
        payoff = product.price_mc(n_sim=10)  # Quick approximation
        payoffs.append(payoff)
    
    payoffs = np.array(payoffs)
    
    # Calculate risk metrics
    mean_payoff = np.mean(payoffs)
    var_95 = PricingEngine.calculate_var(payoffs, 0.95)
    cvar_95 = PricingEngine.calculate_cvar(payoffs, 0.95)
    
    plt.figure(figsize=(12, 6))
    
    # Histogram
    plt.subplot(1, 2, 1)
    plt.hist(payoffs, bins=50, alpha=0.7, color='skyblue', density=True)
    plt.axvline(x=mean_payoff, color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {mean_payoff:.3f}')
    plt.axvline(x=var_95, color='orange', linestyle='--', linewidth=2,
                label=f'VaR 95%: {var_95:.3f}')
    plt.xlabel('Payoff')
    plt.ylabel('Density')
    plt.title(f'Payoff Distribution - {product.__class__.__name__}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Cumulative distribution
    plt.subplot(1, 2, 2)
    sorted_payoffs = np.sort(payoffs)
    y_vals = np.arange(len(sorted_payoffs)) / float(len(sorted_payoffs))
    
    plt.plot(sorted_payoffs, y_vals, 'b-', linewidth=2)
    plt.axhline(y=0.05, color='red', linestyle='--', alpha=0.7, 
                label='5% quantile (VaR 95%)')
    plt.axvline(x=var_95, color='red', linestyle='--', alpha=0.7)
    plt.xlabel('Payoff')
    plt.ylabel('Cumulative Probability')
    plt.title('Cumulative Distribution Function')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print(f"Payoff Statistics for {product.__class__.__name__}:")
    print(f"  Mean: {mean_payoff:.4f}")
    print(f"  Std Dev: {np.std(payoffs):.4f}")
    print(f"  VaR 95%: {var_95:.4f}")
    print(f"  CVaR 95%: {cvar_95:.4f}")
    print(f"  Max: {np.max(payoffs):.4f}")
    print(f"  Min: {np.min(payoffs):.4f}")

# Plot distributions for all products
for product, name in zip(products, names):
    plot_payoff_distribution(product)
    print("\n" + "="*50 + "\n")

## 6. Barrier Hit Probabilities

For barrier products, visualize the probability of barrier events.

In [ ]:
def plot_barrier_analysis(barrier_product):
    """
    Plot barrier hit analysis for barrier products.
    """
    if not hasattr(barrier_product, 'barrier_probability'):
        print("Product does not have barrier functionality")
        return
    
    # Barrier hit probability
    hit_prob = barrier_product.barrier_probability(n_sim=5000)
    print(f"Barrier hit probability: {hit_prob:.1%}")
    
    # Plot sample paths
    barrier_product.plot_barrier_paths(n_paths=20)
    
    # Hit time distribution
    hit_times = []
    n_sim = 1000
    
    np.random.seed(42)
    dt = barrier_product.maturity / 252
    
    for _ in range(n_sim):
        path = [barrier_product.spot]
        hit_time = None
        
        for t in range(1, 253):
            dW = np.random.normal(0, np.sqrt(dt))
            dS = barrier_product.r * path[-1] * dt + barrier_product.vol * path[-1] * dW
            new_price = path[-1] + dS
            path.append(new_price)
            
            if hit_time is None:
                if barrier_product.knock_direction == 'up':
                    if new_price >= barrier_product.barrier:
                        hit_time = t * dt
                else:
                    if new_price <= barrier_product.barrier:
                        hit_time = t * dt
        
        if hit_time is not None:
            hit_times.append(hit_time)
    
    if hit_times:
        plt.figure(figsize=(10, 6))
        plt.hist(hit_times, bins=20, alpha=0.7, color='orange', density=True)
        plt.axvline(x=np.mean(hit_times), color='red', linestyle='--', 
                   label=f'Mean hit time: {np.mean(hit_times):.2f} years')
        plt.xlabel('Time to Barrier Hit (years)')
        plt.ylabel('Density')
        plt.title('Distribution of Barrier Hit Times')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

# Analyze barrier option
barrier_opt = products[2]
plot_barrier_analysis(barrier_opt)

## 7. Range Accrual Efficiency

For range accrual products, show coupon accrual patterns.

In [ ]:
def plot_range_accrual_analysis(range_product):
    """
    Plot range accrual efficiency analysis.
    """
    if not hasattr(range_product, 'calculate_accrual_probabilities'):
        print("Product does not have range accrual functionality")
        return
    
    # Accrual probabilities
    probs = range_product.calculate_accrual_probabilities(n_sim=5000)
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.bar(range(1, len(probs) + 1), probs, alpha=0.7, color='green')
    plt.xlabel('Period')
    plt.ylabel('Full Accrual Probability')
    plt.title('Accrual Probabilities by Period')
    plt.xticks(range(1, len(probs) + 1))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    # Plot ranges
    time_points = np.linspace(0, range_product.maturity, len(range_product.ranges))
    lower_bounds = [r[0] for r in range_product.ranges]
    upper_bounds = [r[1] for r in range_product.ranges]
    
    plt.fill_between(time_points, lower_bounds, upper_bounds, alpha=0.3, color='blue')
    plt.axhline(y=range_product.spot, color='red', linestyle='--', label='Current Spot')
    plt.xlabel('Time (years)')
    plt.ylabel('Price Level')
    plt.title('Accrual Ranges Over Time')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Overall efficiency
    avg_prob = np.mean(probs)
    print(f"Average full accrual probability: {avg_prob:.1%}")
    print(f"Expected total coupon: {avg_prob * sum(range_product.coupon_rates) * range_product.maturity:.1%}")

# Analyze range accrual
range_acc = products[3]
plot_range_accrual_analysis(range_acc)

## Summary

This notebook provides comprehensive visualizations for structured products:

1. **Payoff Diagrams**: Show terminal payoff profiles
2. **Greeks Analysis**: Sensitivity to market parameters  
3. **Scenario Heatmaps**: Price changes under different conditions
4. **Risk Distributions**: Payoff probability distributions with VaR/CVaR
5. **Barrier Analysis**: Hit probabilities and timing for barrier products
6. **Range Accrual**: Coupon accrual efficiency and range visualization

These visualizations help structure product pitches by clearly communicating payoff structures, risks, and market sensitivities to clients.